In [34]:
# ==========================================================
# 📘 BRONZE → SILVER (DELTA LAKE)
# ==========================================================
# Este notebook lê os dados brutos de posição dos ônibus no MinIO (camada Bronze),
# transforma-os em formato tabular e escreve incrementalmente em formato Delta
# na camada Silver.
# ==========================================================

In [35]:
import os
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, explode, current_timestamp, to_timestamp, to_date
from delta.tables import DeltaTable

MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT_DOCKER")
MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER",)
MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD")

BRONZE_PATH = "s3a://bronze/posicao/*/*/*/"
SILVER_PATH = "s3a://silver/posicao/"

print(f"Lendo Bronze de: {BRONZE_PATH}")
print(f"Escrevendo Silver em: {SILVER_PATH}")

Lendo Bronze de: s3a://bronze/posicao/*/*/*/
Escrevendo Silver em: s3a://silver/posicao/


In [36]:
# Inicialização do Spark com suporte ao Delta Lake e MinIO
builder = (
    SparkSession.builder.appName("BronzeToSilver_Delta")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.endpoint", f"http://{MINIO_ENDPOINT}")
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", True)
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

In [37]:
# Definir schema explícito para evitar erro de inferência

schema = StructType([
    StructField("hr", StringType(), True),
    StructField("l", ArrayType(
        StructType([
            StructField("c", StringType(), True),   # código visível (ex: "6L10-10")
            StructField("cl", IntegerType(), True), # código numérico
            StructField("sl", IntegerType(), True), # sentido (1 = ida, 2 = volta)
            StructField("lt0", StringType(), True), # terminal inicial
            StructField("lt1", StringType(), True), # terminal final
            StructField("qv", IntegerType(), True), # quantidade de veículos
            StructField("vs", ArrayType(
                StructType([
                    StructField("p", IntegerType(), True),  # código do veículo
                    StructField("a", BooleanType(), True),  # acessibilidade
                    StructField("ta", StringType(), True),  # timestamp de atualização
                    StructField("py", DoubleType(), True),  # latitude
                    StructField("px", DoubleType(), True),  # longitude
                ])
            ), True)
        ])
    ), True)
])

In [38]:
df_raw = spark.read.option("multiline", True).schema(schema).json(BRONZE_PATH)
df_raw.printSchema()

root
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: integer (nullable = true)
 |    |    |-- sl: integer (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: integer (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- p: integer (nullable = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)



In [39]:
df_exploded = (
    df_raw
    .withColumn("linha", explode(col("l")))
    .withColumn("veiculo", explode(col("linha.vs")))
)

df_clean = (
    df_exploded.select(
        col("linha.c").alias("codigo_linha_texto"),
        col("linha.cl").alias("codigo_linha"),
        col("linha.sl").alias("sentido"),
        col("linha.lt0").alias("terminal_inicial"),
        col("linha.lt1").alias("terminal_final"),
        col("veiculo.p").alias("codigo_veiculo"),
        col("veiculo.a").alias("acessibilidade"),
        to_timestamp(col("veiculo.ta")).alias("ultima_atualizacao"),
        col("veiculo.py").alias("latitude"),
        col("veiculo.px").alias("longitude"),
        to_timestamp(col("hr")).alias("hora_referencia"),
    )
    .dropDuplicates(["codigo_veiculo", "hora_referencia"])
    .withColumn("data_ref", to_date(col("ultima_atualizacao")))
    .withColumn("data_coleta", to_date(col("hora_referencia")))
    .withColumn("data_ingestao", to_date(current_timestamp()))
    .withColumn("ingest_timestamp", current_timestamp())
)
df_clean.show(10)

+------------------+------------+-------+--------------------+-----------------+--------------+--------------+-------------------+-------------------+-------------------+-------------------+----------+-----------+-------------+--------------------+
|codigo_linha_texto|codigo_linha|sentido|    terminal_inicial|   terminal_final|codigo_veiculo|acessibilidade| ultima_atualizacao|           latitude|          longitude|    hora_referencia|  data_ref|data_coleta|data_ingestao|    ingest_timestamp|
+------------------+------------+-------+--------------------+-----------------+--------------+--------------+-------------------+-------------------+-------------------+-------------------+----------+-----------+-------------+--------------------+
|           414P-10|       33873|      2|TERM. NORTE METRÔ...|   VL. INDUSTRIAL|          3117|         false|2025-10-28 00:40:49|        -23.5369965|         -46.563503|2025-10-29 21:41:00|2025-10-28| 2025-10-29|   2025-10-29|2025-10-29 23:44:...|
|   

In [40]:
# Escrita ou merge no Delta Lake

if DeltaTable.isDeltaTable(spark, SILVER_PATH):
    print("⚙️ Atualizando tabela Delta existente...")
    delta_table = DeltaTable.forPath(spark, SILVER_PATH)
    (
        delta_table.alias("t")
        .merge(
            df_clean.alias("s"),
            "t.codigo_veiculo = s.codigo_veiculo AND t.data_ref = s.data_ref"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
else:
    print("🆕 Criando tabela Delta inicial...")
    (
        df_clean
        .write
        .format("delta")
        .mode("append")  # evita sobrescrever
        .partitionBy("codigo_linha", "data_ref")
        .save(SILVER_PATH)
    )

🆕 Criando tabela Delta inicial...


In [41]:
# Verificação de resultado

silver_delta = DeltaTable.forPath(spark, SILVER_PATH)
df_result = silver_delta.toDF()

print("Total de registros na Silver:")
print(df_result.count())

df_result.show(5)

Total de registros na Silver:
68547
+------------------+------------+-------+----------------+--------------------+--------------+--------------+-------------------+------------+-----------+-------------------+----------+-----------+-------------+--------------------+
|codigo_linha_texto|codigo_linha|sentido|terminal_inicial|      terminal_final|codigo_veiculo|acessibilidade| ultima_atualizacao|    latitude|  longitude|    hora_referencia|  data_ref|data_coleta|data_ingestao|    ingest_timestamp|
+------------------+------------+-------+----------------+--------------------+--------------+--------------+-------------------+------------+-----------+-------------------+----------+-----------+-------------+--------------------+
|           342N-10|        1003|      1|   E.T. ITAQUERA|COHAB PRES. JUSCE...|         47311|          true|2025-10-29 23:14:46|-23.56257475|-46.4166955|2025-10-29 20:14:00|2025-10-29| 2025-10-29|   2025-10-29|2025-10-29 23:44:...|
|           342N-10|        1003